Aiming to reacreate the VGG 19 network. <br>
Input is 10,3,224,224 dims in the order B,C,W,H <br>
Preprocessing of images was done by subtracting the mean RGB value, computed on the training set, from each pixel. <BR>

[Architecture](https://gist.github.com/ksimonyan/211839e770f7b538e2d8#file-readme-md) <br>

In [51]:
import os
import torch
from torch import nn,optim
from torch.utils.data import Dataset,DataLoader
from torchvision import io
import pandas as pd
import tqdm
import csv

# Prepare and load the dataset

## Data Preparation

In [48]:
# unzip the file
import zipfile
import shutil


with zipfile.ZipFile("./data/caltech256.zip","r") as zip:
    zip.extractall("./data/caltech256/")
shutil.rmtree("./data/caltech256/256_objectcategories/")
shutil.move("./data/caltech256/256_ObjectCategories/","./data/caltech256/train/")
os.mkdir("./data/caltech256/test/")

In [49]:
# train/test split
for a,_,files in os.walk("./data/caltech256/train/"):
    for f in files[:5]:
        if not os.path.exists("./data/caltech256/test/"+os.path.basename(a)+"/"):
            os.mkdir("./data/caltech256/test/"+os.path.basename(a)+"/") 
        os.rename(a+"/"+f,"./data/caltech256/test/"+os.path.basename(a)+"/"+f)

In [50]:
# train/test labels
train_labels = []
test_labels = []

for dirpath,_,files in os.walk("./data/caltech256/train/"):
    if len(files)>0:
        for f in files:
            path = dirpath+"/"+f
            label = int(os.path.basename(dirpath).split(".")[0])
            train_labels.append([path,label])

with open('./data/caltech256/train_labels.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(train_labels)


for dirpath,_,files in os.walk("./data/caltech256/test/"):
    if len(files)>0:
        for f in files:
            path = dirpath+"/"+f
            label = int(os.path.basename(dirpath).split(".")[0])
            test_labels.append([path,label])

with open('./data/caltech256/test_labels.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(test_labels)

## Datasets and Dataloaders

In [58]:
class ImageDatasetCustom(Dataset):
    def __init__(self,image_labels,transform=None,target_transform=None):
        self.image_labels = pd.read_csv(image_labels)
        self.transform = transform
        self.target_transform = target_transform
    
    def __len__(self):
        return len(self.image_labels)
    
    def __getitem__(self, idx):
        image = io.read_image(self.image_labels.iloc[idx, 0])/255
        label = self.image_labels.iloc[idx, 1]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

train_dataset = ImageDatasetCustom("./data/caltech256/train_labels.csv")
                                
test_dataset = ImageDatasetCustom("./data/caltech256/test_labels.csv")

train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False)
print(len(train_dataset),len(test_dataset))
test_dataset[0]

29322 1284


(tensor([[[0.1059, 0.1098, 0.1059,  ..., 0.6000, 0.6000, 0.5961],
          [0.1020, 0.1020, 0.1020,  ..., 0.6000, 0.6000, 0.5961],
          [0.0941, 0.0941, 0.0941,  ..., 0.6039, 0.6000, 0.6000],
          ...,
          [0.2667, 0.2784, 0.2941,  ..., 0.0784, 0.0784, 0.0784],
          [0.2392, 0.2902, 0.2667,  ..., 0.0784, 0.0784, 0.0784],
          [0.2431, 0.2863, 0.2588,  ..., 0.0784, 0.0784, 0.0784]],
 
         [[0.1176, 0.1216, 0.1176,  ..., 0.7098, 0.7098, 0.7059],
          [0.1137, 0.1137, 0.1137,  ..., 0.7098, 0.7098, 0.7059],
          [0.1059, 0.1059, 0.1059,  ..., 0.7137, 0.7098, 0.7098],
          ...,
          [0.3098, 0.3216, 0.3373,  ..., 0.0588, 0.0588, 0.0588],
          [0.2824, 0.3333, 0.3098,  ..., 0.0588, 0.0588, 0.0588],
          [0.2863, 0.3294, 0.3020,  ..., 0.0588, 0.0588, 0.0588]],
 
         [[0.1373, 0.1412, 0.1373,  ..., 0.7569, 0.7569, 0.7529],
          [0.1333, 0.1333, 0.1333,  ..., 0.7569, 0.7569, 0.7529],
          [0.1255, 0.1255, 0.1255,  ...,